In [29]:
"""
Grad-CAM para explicación de modelos de visión - Versión secuencial explicativa

Este script demuestra cómo utilizar Grad-CAM (Gradient-weighted Class Activation Mapping)
para explicar las predicciones de un modelo de visión ResNet entrenado para clasificación
de imágenes.

Cada sección está comentada para explicar su propósito y funcionamiento.
"""

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import random
import matplotlib.gridspec as gridspec

# ---------------------------------------------------------------------
# SECCIÓN 1: CONFIGURACIÓN INICIAL
# ---------------------------------------------------------------------
print("="*70)
print("GRAD-CAM PARA EXPLICACIÓN DE MODELOS DE VISIÓN - TUTORIAL SECUENCIAL")
print("="*70)

# Configurar Parámetros
IMAGE_PATH = None  # Ejemplo: "../work/val_images_classes/0/imagen.jpg"
CLASS_ID = None  # Opciones: 0, 1, 2, 3, 4 o None
RANDOM_IMAGE = True  

# Rutas y configuración básica
MODEL_PATH = "../work/optuna_temp_artifacts/04 ResNet_1.0.0_20250410_175138.pth"
IMAGE_SIZE = 299
OUTPUT_DIR = "gradcam_results"
VAL_IMAGES_DIR = "../work/val_images_classes"

# Crear directorio para resultados
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Definir nombres de clases
class_names = ['0', '1', '2', '3', '4']

print("\n[PASO 1] Configuración completada.")
print(f"- Modelo a explicar: {MODEL_PATH}")
print(f"- Resultados se guardarán en: {OUTPUT_DIR}")

# ---------------------------------------------------------------------
# SECCIÓN 2: CARGAR EL MODELO
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 2] CARGANDO EL MODELO")
print("="*50)

# Determinar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"- Utilizando dispositivo: {device}")

# Cargar el modelo previamente entrenado
try:
    model = torch.load(MODEL_PATH, map_location=device)
    model.eval()  # Poner el modelo en modo evaluación
    print("- Modelo cargado correctamente.")
except Exception as e:
    print(f"ERROR al cargar el modelo: {e}")
    import sys
    sys.exit(1)

# ---------------------------------------------------------------------
# SECCIÓN 3: DEFINIR FUNCIONES DE PREPROCESAMIENTO
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 3] PREPARANDO FUNCIONES DE PREPROCESAMIENTO")
print("="*50)

# Transformaciones para normalizar las imágenes
val_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

print("- Definidas transformaciones para preprocesamiento de imágenes:")
print("  * Redimensionar a", IMAGE_SIZE, "x", IMAGE_SIZE)
print("  * Convertir a tensor")
print("  * Normalizar con valores ImageNet")

# ---------------------------------------------------------------------
# SECCIÓN 4: IMPLEMENTACIÓN DE GRAD-CAM
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 4] IMPLEMENTANDO GRAD-CAM")
print("="*50)

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        # Registrar hooks
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_backward_hook(self.save_gradient)
    
    def save_activation(self, module, input, output):
        self.activations = output
    
    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]
    
    def forward(self, x):
        return self.model(x)
    
    def backward(self, output, target_class):
        self.model.zero_grad()
        output[:, target_class].backward(retain_graph=True)
    
    def generate_cam(self, x, target_class):
        # Forward pass
        output = self.forward(x)
        
        # Backward pass
        self.backward(output, target_class)
        
        # Obtener pesos de los gradientes
        weights = torch.mean(self.gradients, dim=(2, 3), keepdim=True)
        
        # Calcular el mapa de activación ponderado
        cam = torch.sum(weights * self.activations, dim=1, keepdim=True)
        cam = torch.relu(cam)  # Aplicar ReLU para obtener solo las contribuciones positivas
        
        # Normalizar
        cam = cam - cam.min()
        cam = cam / cam.max()
        
        return cam

def load_image(image_path):
    """Cargar y preprocesar una imagen para predicción con el modelo"""
    try:
        image = Image.open(image_path).convert('RGB')
        return val_transforms(image).unsqueeze(0).to(device), np.array(image)
    except Exception as e:
        print(f"ERROR al cargar la imagen {image_path}: {e}")
        return None, None

def predict_class(image_tensor):
    """Predecir la clase de una imagen con el modelo cargado"""
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        probs = probabilities[0].cpu().numpy()
    
    return predicted_class, probs

def select_random_image(class_id=None):
    """Seleccionar una imagen aleatoria, ya sea de una clase específica o de cualquier clase"""
    if class_id is not None:
        class_dir = os.path.join(VAL_IMAGES_DIR, str(class_id))
        if not os.path.exists(class_dir):
            print(f"ERROR: No se encontró el directorio para la clase {class_id}")
            return None, None
            
        image_files = os.listdir(class_dir)
        if not image_files:
            print(f"ERROR: No hay imágenes en la clase {class_id}")
            return None, None
            
        selected_image = random.choice(image_files)
        return os.path.join(class_dir, selected_image), class_id
    else:
        class_dirs = [os.path.join(VAL_IMAGES_DIR, c) for c in class_names if os.path.exists(os.path.join(VAL_IMAGES_DIR, c))]
        if not class_dirs:
            print("ERROR: No se encontraron directorios de clases")
            return None, None
            
        selected_class_dir = random.choice(class_dirs)
        class_id = int(os.path.basename(selected_class_dir))
        
        image_files = os.listdir(selected_class_dir)
        if not image_files:
            print(f"ERROR: No hay imágenes en {selected_class_dir}")
            return None, None
            
        selected_image = random.choice(image_files)
        return os.path.join(selected_class_dir, selected_image), class_id

def visualize_gradcam(cam, image, class_idx, save_path=None, show=False):
    """Visualizar el mapa de activación Grad-CAM superpuesto sobre la imagen original"""
    # Convertir CAM a numpy y redimensionar
    cam = cam.squeeze().detach().cpu().numpy()
    cam = np.uint8(255 * cam)
    
    # Redimensionar CAM al tamaño de la imagen original
    cam = np.array(Image.fromarray(cam).resize((image.shape[1], image.shape[0]), Image.BILINEAR))
    
    # Crear visualización
    fig = plt.figure(figsize=(15, 5))
    gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05])
    
    # Imagen original
    ax1 = plt.subplot(gs[0])
    ax1.imshow(image)
    ax1.set_title("Imagen Original")
    ax1.axis('off')
    
    # Grad-CAM superpuesto
    ax2 = plt.subplot(gs[1])
    ax2.imshow(image)
    im = ax2.imshow(cam, cmap='jet', alpha=0.5)
    ax2.set_title(f"Grad-CAM Clase {class_idx}")
    ax2.axis('off')
    
    # Barra de color
    ax3 = plt.subplot(gs[2])
    cbar = plt.colorbar(im, cax=ax3)
    cbar.set_ticks([0, 128, 255])
    cbar.set_ticklabels(['Baja', 'Media', 'Alta'])
    cbar.set_label('Contribución a la clase')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=150)
    
    if show:
        plt.show()
    else:
        plt.close()



print("- Funciones definidas:")
print("  * GradCAM: Clase para implementar Grad-CAM")
print("  * load_image: Para cargar y preprocesar imágenes")
print("  * predict_class: Para obtener predicciones del modelo")
print("  * select_random_image: Para seleccionar imágenes aleatorias")
print("  * visualize_gradcam: Para visualizar los resultados de Grad-CAM")

# ---------------------------------------------------------------------
# SECCIÓN 5: SELECCIONAR UNA IMAGEN PARA ANALIZAR
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 5] SELECCIONANDO UNA IMAGEN PARA ANALIZAR")
print("="*50)

true_class = None

if IMAGE_PATH:
    if not os.path.exists(IMAGE_PATH):
        print(f"ERROR: La imagen {IMAGE_PATH} no existe")
        import sys
        sys.exit(1)
    print(f"- Usando imagen proporcionada: {IMAGE_PATH}")
    true_class = None
    
elif CLASS_ID is not None:
    IMAGE_PATH, true_class = select_random_image(CLASS_ID)
    if IMAGE_PATH is None:
        print(f"ERROR: No se pudo seleccionar una imagen de la clase {CLASS_ID}")
        import sys
        sys.exit(1)
    print(f"- Seleccionada imagen aleatoria de clase {CLASS_ID}: {IMAGE_PATH}")
    
else:
    IMAGE_PATH, true_class = select_random_image()
    if IMAGE_PATH is None:
        print("ERROR: No se pudo seleccionar una imagen aleatoria")
        import sys
        sys.exit(1)
    print(f"- Seleccionada imagen aleatoria: {IMAGE_PATH}")

if true_class is not None:
    print(f"- Clase real de la imagen: {true_class}")

# ---------------------------------------------------------------------
# SECCIÓN 6: PREDECIR CLASE DE LA IMAGEN
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 6] PREDICIENDO CLASE DE LA IMAGEN")
print("="*50)

img_tensor, img_original = load_image(IMAGE_PATH)
if img_tensor is None:
    print("ERROR: No se pudo cargar la imagen.")
    import sys
    sys.exit(1)

predicted_class, probabilities = predict_class(img_tensor)

print(f"- Imagen: {os.path.basename(IMAGE_PATH)}")
print(f"- Clase predicha: {predicted_class}")
if true_class is not None:
    if predicted_class == true_class:
        print(f"- ¡Predicción correcta! La clase real es: {true_class}")
    else:
        print(f"- Predicción incorrecta. La clase real es: {true_class}")

print("- Probabilidades por clase:")
for i, prob in enumerate(probabilities):
    print(f"  * Clase {i}: {prob:.4f}")

# ---------------------------------------------------------------------
# SECCIÓN 7: APLICAR GRAD-CAM
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 7] APLICANDO GRAD-CAM")
print("="*50)

# Obtener la capa objetivo (última capa convolucional)
target_layer = None
for module in model.modules():
    if isinstance(module, nn.Conv2d):
        target_layer = module

if target_layer is None:
    print("ERROR: No se encontró una capa convolucional en el modelo")
    import sys
    sys.exit(1)

print(f"- Capa objetivo para Grad-CAM: {target_layer}")

# Crear instancia de Grad-CAM
gradcam = GradCAM(model, target_layer)

# Generar mapas de activación para cada clase
print("\n- Generando mapas de activación para cada clase...")
for class_idx in range(len(class_names)):
    # Generar CAM para esta clase
    cam = gradcam.generate_cam(img_tensor, class_idx)
    
    # Visualizar y guardar
    save_path = os.path.join(OUTPUT_DIR, f"{os.path.basename(IMAGE_PATH.split('.')[-2])}_gradcam_class_{class_idx}.jpg")
    visualize_gradcam(cam, img_original, class_idx, save_path=save_path)
    print(f"  * Guardado Grad-CAM para clase {class_idx} en: {save_path}")

# ---------------------------------------------------------------------
# SECCIÓN 8: EXPLICACIÓN DE LOS RESULTADOS
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 8] EXPLICACIÓN DE LOS RESULTADOS")
print("="*50)

print("""
¿Qué significan los resultados de Grad-CAM?

1. Las áreas ROJAS indican las regiones más importantes para la predicción de cada clase.
2. Las áreas AZULES indican regiones menos relevantes para la predicción.
3. La intensidad del color muestra la importancia relativa de cada región.

Grad-CAM nos permite:
1. Entender en qué partes de la imagen se enfoca el modelo para cada clase
2. Verificar si el modelo está usando las características correctas
3. Identificar posibles sesgos o problemas en el modelo
4. Mejorar la interpretabilidad de las predicciones
""")

# ---------------------------------------------------------------------
# SECCIÓN 9: CONCLUSIÓN
# ---------------------------------------------------------------------
print("\n" + "="*50)
print("[PASO 9] CONCLUSIÓN")
print("="*50)

print(f"""
El análisis Grad-CAM nos ha permitido visualizar las regiones de la imagen que más
influyen en la predicción de cada clase. La clase predicha fue {predicted_class}
con una probabilidad de {probabilities[predicted_class]:.4f}.

Todas las visualizaciones se han guardado en:
{OUTPUT_DIR}

Grad-CAM es una herramienta poderosa para:
1. Entender el comportamiento del modelo
2. Validar si el modelo está aprendiendo patrones significativos
3. Identificar posibles problemas de generalización
4. Mejorar la confianza en las predicciones del modelo
""")

print("\n" + "="*70)
print("FIN DEL TUTORIAL")
print("="*70)

GRAD-CAM PARA EXPLICACIÓN DE MODELOS DE VISIÓN - TUTORIAL SECUENCIAL

[PASO 1] Configuración completada.
- Modelo a explicar: ../work/optuna_temp_artifacts/04 ResNet_1.0.0_20250410_175138.pth
- Resultados se guardarán en: gradcam_results

[PASO 2] CARGANDO EL MODELO
- Utilizando dispositivo: cuda
- Modelo cargado correctamente.

[PASO 3] PREPARANDO FUNCIONES DE PREPROCESAMIENTO
- Definidas transformaciones para preprocesamiento de imágenes:
  * Redimensionar a 299 x 299
  * Convertir a tensor
  * Normalizar con valores ImageNet

[PASO 4] IMPLEMENTANDO GRAD-CAM
- Funciones definidas:
  * GradCAM: Clase para implementar Grad-CAM
  * load_image: Para cargar y preprocesar imágenes
  * predict_class: Para obtener predicciones del modelo
  * select_random_image: Para seleccionar imágenes aleatorias
  * visualize_gradcam: Para visualizar los resultados de Grad-CAM

[PASO 5] SELECCIONANDO UNA IMAGEN PARA ANALIZAR
- Seleccionada imagen aleatoria: ../work/val_images_classes/3/281e2d7e8-1.jpg
- C